In vector search we have a set of vectors and a query vector and we will find the most similar items in our set for the query.We call this as nearest neighbour search.

**K Nearest Neighbours**

KNN is an effective way to find nearest vectors in space given query vector. It will find K nearst neighbours to the given query vector. But  to find the nearest vectors for our query we will have to calculate its distance with every vector we have in our database. So for database with millions of rows it is a problem.

**Approximate Neighbor Search**

To reduce this computational complexity we can use appproximate neighbour search.It helps for a massive perfomnce improvement in larger datasets. Instead of checking distances between each vector in the database, we will have a good guess of the nearest neighbour.In ANN we build index structures which narrow down search space and improve lookup times.There are many ANN algorithms available. Some of them are

* HNSW
* IVFLAT

# HNSW (Hierarchical Navigable Small World)


HNSW is one of the most common indexing method. It uses hierarchical graph architecture to index vectors.HNSW is formed by combining two algorithms

* Skip Lits
* Navigable small world


### Skip List

Skip list is based on Linked List  data structure or we can say it is an extention of linked list data structure. We know linked list forms a series of connected node where each node stores the data and address of next node.

<!-- <img src="data/linked_list.png"> -->

A skip list maintains a layered linked list architecture where top layer has the longest links between the elements and it reduces exponentially as we move down.


<img src="images_tmp/link_list1.png">

In normal linked list, Initially it contain n elements in sorted manner(ascending).Suppose we want to search '11'.Now we will traverse from left to right and find the matching element.We will hop from left starting from 2 till we reach ll. We can see the time complexity is high here.It will be o(n). This is the problem the skip list will solve.

Lets see how skip list is constructed.We know skip list has various levels starting from 0 to k. At Level 0, we have a linked list as before. Now starting from first element '2', We will choose a random probability P(S). If P(S) >= 0.5 we will increase the hight of that node(ie we extends to layer 1).If P(S)<0.5 we will hop to next node. We can assume this as tosing a coin. If we get heads we will increase the height of node or else hop to next node on same layer.


Lets start from 2, and flip a coin.Suppose we got head.Now we grow that node to layer 1. Next move to 8, again flip the coin.suppose we got tails now. So we dont grow the node and move to next node.Now we reach 11.Suppose we got heads by fliping the coin.So we will grow the node.

<img src="images_tmp/link_list2.png">

<img src="images_tmp/link_list3.png">

<img src="images_tmp/link_list4.png">

In this manner we will traverse through all elements in Layer 0 and grow our nodes. 

<img src="images_tmp/link_list5.png">

Next we will start hoping from left to right in Layer2 in similar manner we did in Layer 0 and grow Layer 3.Finally based on random probabilities we will end up something like this

<img src="images_tmp/link_list6.png">

Here we have four layers - L0,L1,L2 and L3. Number of layers depends up on the user. But a thump of rule is to use `log n` number of layers.


How Search will Occur?

Suppose user wants to search key 20. 

- First it will start from upper most layer Layer 4. First node it inspect is 2. 20 is greater than 2. So it hopes to next node in layer 4 which is 13.Since 20 > 13 and we dont have nodes after it in layer 4, we will move to layer 3 with node as 13.

- Next we will hope to next element in Layer 3 which is 26. 20 < 26, so we hope one layer below under 13. Now in Layer 2 we move right to reach 19. since 20 > 19  we again hop to next node and reaches 20.


<img src="images_tmp/link_list7.png">

This is how a skip list will work.


### Navigable small world

In NSW we have a graph like structure in which each vertex is connected to several other vertices.We call these connected vertices friends, and each vertex keeps a friend list, creating our graph.

When searching an NSW graph, we begin at a pre-defined entry-point. This entry point connects to several nearby vertices.So when we have quey vector reaching a entry point it finds the distance to all vertices in its friend list and will hop to next closest vertex. 

This greedy search process continous by moving from vertex to vertex until it found no nearer vertices than our current vertex — this is a local minimum and acts as our stopping condition.
<img src="images_tmp/nsw1.webp" width=500 heigh=300>

Actually while routing we passes through 2 phases:

- A zoom out phase where we pass through low degree vertices(degree is the number of links a vertex has)
- A zoom in phase where we pass through high degree vertices
<img src="images_tmp/nsw2.webp" width=500 heigh=300>

Our stopping condition is finding no nearer vertices in our current vertex’s friend list. Because of this, we are more likely to hit a local minimum and stop too early when in the zoom-out phase (fewer links, less likely to find a nearer vertex).To minimize this probability of early stopping we can increase the average degree of vertices.But it will increases time complexity. So we have to balance the average degree of vertices between recall and search speed.

### Towards HNSW

HNSW is a combination of skip list and navigable small world.Adding hierarchy to NSW produces a graph where links are separated across different layers.At the top layer, we have the longest links, and at the bottom layer, we have the shortest.

<img src="images_tmp/hnsw.webp" width=500 heigh=300>

While searching,we start from a pre-defined point at the uppermost layer of the hierarchy and greedily traverse the graph to find the closest element to the target data point in that layer. Once we reach the nearest node, we descend to the layer below and repeat the process until “K” nearest neighbors of the target node are found.



<img src="images_tmp/hnsw2.webp">


**Pros**

* Needs high memory: HNSW maintains a hierarchical structure of embeddings, which significantly increases memory consumption compared to algorithms like NSW. 
* Parameter tuning: Careful parameter tuning is required for good perfomance

**parameters**

* EF Construction: This parameter is used when the graph is being built. Think of it as the algorithm's thoroughness when it's adding a new point to the graph. A higher EF Construction means the algorithm will search more extensively for neighbors, which can make the graph more accurate. However, this also means it will take more time and resources to build the graph.
* EF Search: This parameter comes into play when you're searching for the nearest neighbors of a specific point in the graph. A higher EF Search means the algorithm will look more extensively for nearest neighbors, which can improve the accuracy of the search. However, this might slow down the search process.

* M: This parameter determines the maximum number of connections (or edges) a node (or data point) can have in the graph. It influences the density and navigability of the graph, affecting both the build and search phases.

```
CREATE INDEX ON items USING hnsw (embedding vector_cosine_ops) WITH (m = 16, ef_construction = 64);
```



```
SET hnsw.ef_search = 100;
```